# Banana World Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Import Dependencies

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
from unityagents import UnityEnvironment

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [2]:
env = UnityEnvironment(file_name="/home/sebastian/udacity_drl/drlnd-navigation-project/Banana_Linux/Banana.x86_64")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [3]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
random_score = 0                                   # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action at random
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    random_score += reward                         # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score with random actions: {}".format(random_score))

Score with random actions: 0.0


### 4a. Create a Q-Network

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [4]:
class QNetwork(nn.Module):
    
    def __init__(self, num_states=37, num_actions=4, layer_dims=[128, 64, 32]):
        """
        Initialize the Q-Network structure
        """
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(num_states, layer_dims[0])
        self.fc2 = nn.Linear(layer_dims[0], layer_dims[1])
        self.fc3 = nn.Linear(layer_dims[1], layer_dims[2])
        self.fc4 = nn.Linear(layer_dims[2], num_actions)
        
    def forward(self, state):
        """ 
        Forward pass through the Q-Network, which accepts states
        and returns action values.
        """
        
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

### 4b. Create a DQN Agent

Here we will create a DQN Agent that learns using Q-Learning

In [5]:
# Set the torch compute device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class DQNAgent():
    """
    Defines a Deep Q-Network (DQN) Reinforcement Learning Agent
    """

    def __init__(self, action_size, batch_size=64, gamma=0.99,
                 learn_rate=1e-3, learn_period=4, 
                 replay_buffer_size=200000, tau=1e-3, seed=0):
        """Initialize a Reinforcement Learning Agent """
        
        # Unpack arguments
        self.action_size = action_size
        self.batch_size = batch_size
        self.gamma = gamma
        self.learn_period = learn_period
        self.tau = tau
        torch.manual_seed(seed)
        
        # Define the local and target Q-Networks
        self.qnetwork_local = QNetwork().to(device)
        self.qnetwork_target = QNetwork().to(device)
        
        # Define an optimizer for training the Q-Network
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=learn_rate)

        # Create a replay buffer
        self.replay_buffer = ReplayBuffer(action_size, replay_buffer_size, batch_size, seed)
        
        # Initialize quantities to keep track of
        self.t_step = 0
        
    
    def step(self, state, action, reward, next_state, done):
        """ Steps the agent given the results of stepping the RL environment """
        
        # Save experience in replay memory
        self.replay_buffer.add(state, action, reward, next_state, done)
        
        # Learn every LEARN_PERIOD time steps.
        self.t_step = (self.t_step + 1) % self.learn_period
        if self.t_step == 0:
            # Sample a subset of experiences from the buffer as soon 
            # as there are enough experiences to fill a batch
            if len(self.replay_buffer) >= self.batch_size:
                batch = self.replay_buffer.sample()
                self.learn(batch)
               

    def get_action(self, state, epsilon):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        
        # Run a forward pass of the Q-Network given an input state
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > epsilon:
            action = np.argmax(action_values.cpu().data.numpy())
        else:
            action = random.choice(np.arange(self.action_size))
        
        return action
        

    def learn(self, experiences):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states 
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Compute and minimize the loss
        loss = F.mse_loss(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Finally, update the target networks using the new parameters after the training step
        self.soft_update()
        

    def soft_update(self):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(self.qnetwork_target.parameters(), self.qnetwork_local.parameters()):
            target_param.data.copy_(self.tau*local_param.data + (1.0-self.tau)*target_param.data)

            
from collections import deque, namedtuple
class ReplayBuffer:
    """
    Fixed-size buffer to store experience tuples for reinforcement learning
    This replay buffer implementation is directly from the following source:
    https://github.com/udacity/deep-reinforcement-learning/blob/master/dqn/solution/dqn_agent.py
    """

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### 4c. Train the Q-Network

TBD

In [7]:
do_train = False

#######################
### HYPERPARAMETERS ###
#######################
# Training parameters
NUM_EPISODES = 2000         # Total number of episodes to train
AVERAGE_WINDOW = 50         # Number of episodes to compute average reward
MAX_EPISODE_STEPS = 10000   # Maximum number of steps per episode
LEARN_PERIOD = 4            # Number of simulation steps between running a training step
BATCH_SIZE = 64             # Training batch size (sampled from replay buffer)
REPLAY_BUFFER_SIZE = 200000 # Number of experiences in replay buffer for sampling at training time

# Reinforcement Learning parameters
GAMMA = 0.999               # Discount factor
TAU = 1e-3                  # Update rate for the target network

# Learning rate parameters
LEARN_RATE = 1e-3               # Base learning rate
LEARN_RATE_DECAY_FACTOR = 0.25  # Stepwise factor for learning rate
LEARN_RATE_DECAY_PERIOD = 500   # Number of episodes before stepping down learning rate

# Exploration vs. exploitation parameters (epsilon = probability of randomly picking action)
EPS_INIT = 0.5                            # Initial value of epsilon
EPS_FINAL = 0.05                          # Final (and minimum) value of epsilon
EPS_DECAY_STEPS = int(0.9*NUM_EPISODES)   # Number of episodes over which epsilon is linearly decayed to its minimum

# Random seed for reproducibility
RANDOM_SEED = 0

### END HYPERPARAMETERS ###

# Create the agent and learning rate decay scheduler
agent = DQNAgent(action_size, learn_rate=LEARN_RATE, learn_period=LEARN_PERIOD, 
                 batch_size=BATCH_SIZE, replay_buffer_size=REPLAY_BUFFER_SIZE,
                 gamma=GAMMA, tau=TAU, seed=RANDOM_SEED)
lr_decay = optim.lr_scheduler.StepLR(agent.optimizer, 
    step_size=LEARN_RATE_DECAY_PERIOD, gamma=LEARN_RATE_DECAY_FACTOR)

# Train the agent
if do_train:
    epsilon = EPS_INIT
    scores = []
    scores_avg = []
    scores_window = deque(maxlen=AVERAGE_WINDOW)
    for idx in range(NUM_EPISODES):

        # Start the episode by resetting the environment
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        done = False
        score = 0

        # Run the episode to completion
        t_step = 0
        while not done:
            # Get the next action from the agent using the epsilon-greedy policy
            action = agent.get_action(state, epsilon)

            # Step the environment and unpack the information
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0] 

            # Step the agent and learning rate scheduler
            agent.step(state, action, reward, next_state, done)

            # Update the next state and score
            state = next_state
            score += reward
            t_step += 1
            if t_step >= MAX_EPISODE_STEPS:
                break

        # Once the episode is done, decay epsilon and learning rate
        epsilon = max(EPS_FINAL, epsilon - (EPS_INIT-EPS_FINAL)*(idx/EPS_DECAY_STEPS))
        lr_decay.step()

        # Update the average scores
        scores.append(score)
        scores_window.append(score)
        if (idx+1) % AVERAGE_WINDOW == 0:
            mean_score = np.mean(scores_window)
            scores_avg.append(mean_score)
            print("\rAverage score at episode {}: {}".format(idx+1, mean_score))

    ### End training loop ###
            
    # Plot the final scores
    fig = plt.figure()
    plt.plot(np.arange(len(scores)), scores)
    plt.plot(np.arange(len(scores_avg))*AVERAGE_WINDOW, scores_avg)
    plt.xlabel("Episode number")
    plt.ylabel("Total reward")
    plt.legend(["Raw scores", "Average scores"])
    plt.show()
    
    # Save the trained agent to file
    torch.save(agent.qnetwork_local.state_dict(),"trained_weights.pth")

# Else load a pretrained model
else:
    trained_agent = torch.load("trained_weights.pth")
    agent.qnetwork_local.load_state_dict(trained_agent)

### 5. Test the Trained Agent

Now we will run the environment with the trained agent using a fully greedy policy and compare it to taking random actions.

In [9]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    
    # Get an action with the agent, using the fully greedy policy
    action = agent.get_action(state, 0)            
    
    # Then, step the agent using this action
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0]
    done = env_info.local_done[0]
    score += reward
    state = next_state
    if done:
        break
    
print("Score with random agent: {}".format(random_score))
print("Score with trained agent: {}".format(score))

Score with random agent: 0.0
Score with trained agent: 18.0
